In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import os
# path = ""
# path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
# os.chdir(path)
# import time
# FOLDERNAME = "OPUS-OpenNMT-sent2vec-0.85-" + str(time.strftime("%Y%m%d-%H%M"))
# !mkdir $FOLDERNAME

# path = path + FOLDERNAME
# os.chdir(path)
# !pwd

import os
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/OPUS-OpenNMT-sent2vec-0.85-20210412-2246'
os.chdir(path)
!pwd

/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/OPUS-OpenNMT-sent2vec-0.85-20210412-2246


In [3]:
!nvidia-smi

Mon Apr 12 22:47:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 8.5MB/s 
     |████████████████████████████████| 2.6MB 34.8MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=a61f626d04ab2c812ecaca07878436d95bc51f84795cbfb2b548eda09e78104b
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 6.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_sent2vec.tar.gz
!mkdir data_bin
!tar -xvf 'opus_sent2vec.tar.gz'

--2021-04-12 22:47:17--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_sent2vec.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94255466 (90M) [application/octet-stream]
Saving to: ‘opus_sent2vec.tar.gz’

opus_sent2vec.tar.g 100%[===================>]  89.89M  65.8MB/s    in 1.4s    

2021-04-12 22:47:21 (65.8 MB/s) - ‘opus_sent2vec.tar.gz’ saved [94255466/94255466]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [6]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train_EM_0.85' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-12 22:47:29,715 INFO] Extracting features...
[2021-04-12 22:47:29,721 INFO]  * number of source features: 0.
[2021-04-12 22:47:29,722 INFO]  * number of target features: 0.
[2021-04-12 22:47:29,722 INFO] Building `Fields` object...
[2021-04-12 22:47:29,722 INFO] Building & saving training data...
[2021-04-12 22:47:31,058 INFO] Building shard 0.
[2021-04-12 22:48:03,626 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-12 22:48:22,218 INFO]  * tgt vocab size: 50004.
[2021-04-12 22:48:22,557 INFO]  * src vocab size: 50002.
[2021-04-12 22:48:23,233 INFO] Building & saving validation data...
[2021-04-12 22:48:23,831 INFO] Building shard 0.
[2021-04-12 22:48:26,144 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-12 22:48:32,706 INFO]  * src vocab size = 50002
[2021-04-12 22:48:32,706 INFO]  * tgt vocab size = 50004
[2021-04-12 22:48:32,706 INFO] Building model...
[2021-04-12 22:48:40,927 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feat

In [8]:
!ls -al model model/

model:
total 43401240
-rw------- 1 root root 1481428927 Apr 13 01:04 en-vi_step_10000.pt
-rw------- 1 root root 1481428927 Apr 12 23:02 en-vi_step_1000.pt
-rw------- 1 root root 1481428927 Apr 13 01:18 en-vi_step_11000.pt
-rw------- 1 root root 1481428927 Apr 13 01:31 en-vi_step_12000.pt
-rw------- 1 root root 1481428927 Apr 13 01:45 en-vi_step_13000.pt
-rw------- 1 root root 1481428927 Apr 13 01:59 en-vi_step_14000.pt
-rw------- 1 root root 1481428927 Apr 13 02:12 en-vi_step_15000.pt
-rw------- 1 root root 1481428927 Apr 13 02:26 en-vi_step_16000.pt
-rw------- 1 root root 1481428927 Apr 13 02:39 en-vi_step_17000.pt
-rw------- 1 root root 1481428927 Apr 13 02:53 en-vi_step_18000.pt
-rw------- 1 root root 1481428927 Apr 13 03:07 en-vi_step_19000.pt
-rw------- 1 root root 1481428927 Apr 13 03:20 en-vi_step_20000.pt
-rw------- 1 root root 1481428927 Apr 12 23:15 en-vi_step_2000.pt
-rw------- 1 root root 1481428927 Apr 13 03:34 en-vi_step_21000.pt
-rw------- 1 root root 1481428927 Apr 13 0

In [9]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-13 05:37:14,673 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [50], which does not match the required output shape [10, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an ou

In [ ]:
!tail vi_test

In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

In [12]:
!ls -al

total 355330
drwx------  2 root root     4096 Apr 12 22:47 data_bin
-rw-------  1 root root  3318349 Apr 12 02:53 en_test
-rw-------  1 root root 26563375 Apr 12 02:53 en_train
-rw-------  1 root root 31608088 Apr 12 03:46 en_train_EM_0.8
-rw-------  1 root root 29845474 Apr 12 03:46 en_train_EM_0.85
-rw-------  1 root root 28377219 Apr 12 03:46 en_train_EM_0.9
-rw-------  1 root root 27245135 Apr 12 03:46 en_train_EM_0.95
-rw-------  1 root root 11930822 Apr 12 03:46 en_train_EM_factor_0.8
-rw-------  1 root root 11270762 Apr 12 03:46 en_train_EM_factor_0.85
-rw-------  1 root root 10719696 Apr 12 03:46 en_train_EM_factor_0.9
-rw-------  1 root root 10293086 Apr 12 03:46 en_train_EM_factor_0.95
-rw-------  1 root root  6897191 Apr 12 03:46 en_train_EM_score_0.8
-rw-------  1 root root  6897191 Apr 12 03:46 en_train_EM_score_0.85
-rw-------  1 root root  6897191 Apr 12 03:46 en_train_EM_score_0.9
-rw-------  1 root root  6897191 Apr 12 03:46 en_train_EM_score_0.95
-rw-------  1 root ro

In [3]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 16.49, 42.1/23.5/14.4/9.7 (BP=0.855, ratio=0.865, hyp_len=655694, ref_len=758454)
